In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedShuffleSplit
MAX_SENTENCE_LENGTH = 400
import nltk
import numpy as np
import torch
from torch.utils.data import Dataset
from nltk.stem.porter import *

In [2]:
stop_words = ["i","i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","to","from","up","down","in","on","off","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","other","some","such","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"]

In [3]:
import spacy
tokenize = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()

In [16]:
test_pos = "/home/cvh255/nlp_hw1/aclImdb/test/pos/"
test_neg = "/home/cvh255/nlp_hw1/aclImdb/test/neg/"

In [17]:
test_pos_files = os.listdir(test_pos)
for f in range(len(test_pos_files)):
    test_pos_files[f] = test_pos + test_pos_files[f]

In [18]:
test_neg_files = os.listdir(test_neg)
for f in range(len(test_neg_files)):
    test_neg_files[f] = test_neg + test_neg_files[f]

In [19]:
test_pos_labels = [1]*len(test_pos_files)
test_neg_labels = [0]*len(test_neg_files)

In [20]:
df = pd.DataFrame(columns=["file_names","labels"])
df["file_names"] = test_pos_files + test_neg_files
df["labels"] = test_pos_labels + test_neg_labels

In [25]:
def get_data(df):
    all_txt = []
    for i,j in df.iterrows():
        f = open(j["file_names"])
        txt = f.read()
        all_txt.append(txt)
    df["content"] = all_txt
    return df

In [26]:
test_df = get_data(df)

In [27]:
from nltk import ngrams

In [28]:
import string
punctuations = string.punctuation
def tokenize1(phrase):
    tokens = tokenize(phrase)
    return [token.text.lower() for token in tokens if (token.text not in punctuations and token.text not in stop_words)]
    

In [32]:
def tokenize_dataset1(dataset,n_gram):
    token_dataset = []
    all_tokens = []
    for sample in dataset:
        tokens = tokenize1(sample)
        bg = list(nltk.bigrams(tokens))
        tg = list(nltk.trigrams(tokens))
        fg = list(ngrams(tokens,4))
        bg_t = [' '.join(a) for a in bg]
        tg_t = [' '.join(a) for a in tg]
        fg_t = [' '.join(a) for a in fg]
        tokens = tokens + bg_t + tg_t + fg_t
        token_dataset.append(tokens)
        all_tokens+=tokens
    return token_dataset, all_tokens

In [33]:
test_df["tokenized1"],_ = tokenize_dataset1(test_df['content'],1)

In [34]:
import pickle
all_train_tokens = pickle.load(open("all_tokens5",'rb'))

In [35]:
from collections import Counter

max_vocab_size = 100000
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id, id2token = build_vocab(all_train_tokens)

In [36]:
import random
random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 50494 ; token i 'm sad
Token i 'm sad; token id 50494


In [37]:
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

test_df['token_idized'] = token2index_dataset(test_df['tokenized1'])

In [42]:
BATCH_SIZE = 100
test_dataset = IMDBDataset(test_df)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                           batch_size = BATCH_SIZE,
                                           collate_fn = pad_fun,
                                           shuffle = True)

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class BagOfWords(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(BagOfWords, self).__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx = 0)
        self.linear = nn.Linear(emb_dim,2)
    
    def forward(self, data, length):
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
        out = self.linear(out.float())
        return out

In [49]:
class IMDBDataset_test(Dataset):
    def __init__(self, csv_file):
        self.data_frame = csv_file

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        file_name = self.data_frame.iloc[idx]["file_names"]
        token_idx = self.data_frame.iloc[idx]["token_idized"]
        label = self.data_frame.iloc[idx]['labels']
        return [token_idx, len(token_idx), label,file_name]


def pad_fun_test(batch):
    data_list = []
    label_list = []
    length_list = []
    file_names = []
    for datum in batch:
        
        label_list.append(datum[2])
        length_list.append(datum[1])
        file_names.append(datum[3])
    for datum in batch:
        if datum[1]>MAX_SENTENCE_LENGTH:
            padded_vec = np.array(datum[0][:MAX_SENTENCE_LENGTH])
        else:
            padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH - datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.from_numpy(np.array(length_list)), torch.from_numpy(np.array(label_list)),np.array(file_names)]

test_dataset_test = IMDBDataset_test(test_df)
test_loader_test = torch.utils.data.DataLoader(dataset = test_dataset_test, 
                                           batch_size = BATCH_SIZE,
                                           collate_fn = pad_fun_test,
                                           shuffle = True)

# Evaluation on Test set

In [52]:
mod_saved = torch.load("model23_tokenize2")

In [81]:
correct = 0
total = 0

running_loss = 0
running_total = 0

criterion = torch.nn.CrossEntropyLoss()
mod_saved.train(False)

for (data, lengths, labels, file_n) in test_loader_test:
    data_batch, length_batch, label_batch = data.cuda(), lengths.cuda(), labels.cuda()
    outputs = mod_saved(data_batch, length_batch)
    outputs = F.softmax(mod_saved(data_batch, length_batch), dim=1)
    loss = criterion(outputs, label_batch)
    predicted = outputs.max(1, keepdim=True)[1]
    mask =(predicted.squeeze(1).eq(label_batch)).cpu().data.numpy()==0
    fns = file_n[mask]
    tns = file_n[np.array(mask)==False]
    actual_out = labels.data.numpy()[mask]
    actual_true_out = labels.data.numpy()[np.array(mask)==False]
    pred_false = predicted.cpu().data.numpy()[mask]
    pred_true = predicted.cpu().data.numpy()[np.array(mask)==False]
    running_loss += loss.data[0] * labels.size(0)
    running_total += labels.size(0)

    total += labels.size(0)
    correct += predicted.eq(labels.view_as(predicted).cuda()).sum().item()
accuracy = 100 * correct / total
epoch_loss = running_loss/running_total

/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [69]:
print("Accuracy on test set is = ",accuracy,"%")
print("Cross Entropy loss on test set is = ",epoch_loss.item())

Accuracy on test set is =  89.2 %
Cross Entropy loss on test set is =  0.43386340141296387


# Results :
## Accuracy on test set is =  89.2 %
## Cross Entropy loss on test set is =  0.43386340141296387

# Incorrect Predictions on Test Set

In [84]:
for i in range(len(fns)):
    print("predicted",pred_false[i][0])
    print("Actual",actual_out[i])
    print(test_df[test_df['file_names'] ==fns[i]]["file_names"].values[0])
    f = open(test_df[test_df['file_names'] ==fns[i]]["file_names"].values[0])
    print(f.read())
    print()
    print("-"*100)

predicted 0
Actual 1
/home/cvh255/nlp_hw1/aclImdb/test/pos/2615_10.txt
Charmed was awesome!!!! I don't get how Pheobe goes to the underworld and makes a deal with the source but then in season 4 is back... how does she get back. is there a deleted episode that was never showed?????? i am confused i brought 1 2 3 4 5 and season 8 but am still confused will someone help me help help help h e l p<br /><br />h e l p<br /><br />h e l p <br /><br />h e l p me me me me lull lull Lilllie loll<br /><br />loll loll lull loll Lilla Lilla loll lull<br /><br />Lilllie<br /><br />Lilla loll Lilla Lilla lull loll lull Lilllie Lilllie ll Lilllie lull

----------------------------------------------------------------------------------------------------
predicted 1
Actual 0
/home/cvh255/nlp_hw1/aclImdb/test/neg/9200_3.txt
I really enjoyed the film. It was really cheesy at times. (They destroy the villain with hair driers--but where are they plugged in?) It's a unique film though, and I enjoyed the acting

# Correct Predictions on Test Set

In [85]:
for i in range(len(tns)-5,len(tns)):
    print("predicted",pred_true[i][0])
    print("Actual",actual_true_out[i])
    print(test_df[test_df['file_names'] == tns[i]]["file_names"].values[0])
    f = open(test_df[test_df['file_names'] == tns[i]]["file_names"].values[0])
    print(f.read())
    print()
    print("-"*100)

predicted 0
Actual 0
/home/cvh255/nlp_hw1/aclImdb/test/neg/10930_1.txt
This film is by far the worst film I have ever seen in my life. A woman "The EX" pretends to be a number of people in order to gain access to her ex-husband. Killing people for no- reason in baths to achieve her goal. The women I don't think ever went to acting college. She just spends the whole film making stupid expressions, and she looks like she is trying her absolute hardest to avoid looking into the camera. Failing on most occasions. She makes friends with her Ex Husband's wife and son, she does this to use them against her husband. At first when you watch this film, you think that the "Ex" wants to kill her ex-husband, maybe because he has treated her in a bad way. But in fact the women is obsessed with the man and wants him back. The two of them used to enjoy rough entertainment (using whips of course). My advice to the general public is do not buy this film, do not rent this film and do not watch it like I 

# Incorrect Predictions on Validation Set

In [89]:
val_df = pd.read_csv("val_df_tok5.csv")
val_df['token_idized'] = token2index_dataset(val_df['tokenized1'])

In [90]:
val_dataset_test = IMDBDataset_test(val_df)
val_loader_test = torch.utils.data.DataLoader(dataset = val_dataset_test, 
                                           batch_size = BATCH_SIZE,
                                           collate_fn = pad_fun_test,
                                           shuffle = True)


In [92]:
data, lengths, labels, file_n = next(iter(val_loader_test))
data_batch, length_batch, label_batch = data.cuda(), lengths.cuda(), labels.cuda()
outputs = mod_saved(data_batch, length_batch)
outputs = F.softmax(mod_saved(data_batch, length_batch), dim=1)
predicted = outputs.max(1, keepdim=True)[1]
mask =(predicted.squeeze(1).eq(label_batch)).cpu().data.numpy()==0
fns = file_n[mask]
actual_out = labels.data.numpy()[mask]
pred_false = predicted.cpu().data.numpy()[mask]
tns = file_n[np.array(mask)==False]
actual_true_out = labels.data.numpy()[np.array(mask)==False]
pred_true = predicted.cpu().data.numpy()[np.array(mask)==False]

In [95]:
for i in range(len(fns)-4,len(fns)):
    print("predicted",pred_false[i][0])
    print("Actual",actual_out[i])
    print(val_df[val_df['file_names'] ==fns[i]]["file_names"].values[0])
    f = open(val_df[val_df['file_names'] ==fns[i]]["file_names"].values[0])
    print(f.read())
    print()
    print("-"*100)

predicted 0
Actual 1
/home/cvh255/nlp_hw1/aclImdb/train/pos/7671_10.txt
Dr. Mordrid, what can I say? Jeffrey Combs has done it again!<br /><br />Anton Mordrid has been on Earth for 100 years waiting for Kabal, an evil sorcerer, to come so he can kill him. Mordrid and Kabal used to train together as kids, so Mordrid knows all Kabal's tricks.<br /><br />The film as a little bit confusing at begin with, but soon you feel a part of the action. I won't give away the ending, so go and watch Doctor Mordrid!<br /><br />I found the film to be very enjoyable because it doesn't have a lot of violence in, nor sexual scenes. The film focused on the plot and that's what I like! I find the best films are the ones where times seems to fly by. This is because you are so engrossed in the film. Doctor Mordrid is a fantasticly engrossing movie! I give it a 20 out of 10. Worth seeing!<br /><br />

----------------------------------------------------------------------------------------------------
predicted

# Correct Predictions on Validation Set

In [96]:
for i in range(len(tns)-5,len(tns)):
    print("predicted",pred_true[i][0])
    print("Actual",actual_true_out[i])
    print(val_df[val_df['file_names'] == tns[i]]["file_names"].values[0])
    f = open(val_df[val_df['file_names'] == tns[i]]["file_names"].values[0])
    print(f.read())
    print()
    print("-"*100)

predicted 0
Actual 0
/home/cvh255/nlp_hw1/aclImdb/train/neg/11611_3.txt
Lady in Cement - PI spoof with ole Blue Eyes.Frank Sinatra is a shamus on a houseboat in Miami in this rarely funny "comedy".Burdened by an annoying and repetitious Hugo Montenegro score and bunch of misfiring punchlines this 1968 flick just never rises above slightly too bawdy to be on TV made for television movie status.Dan Blocker is effective in the Mike Mazurski/Ted De Corsia big galoot role and Raquel Welch should thank her personal trainer.The only thing that makes the DVD worth keeping or seeing is the collection of cheesy trailers for Welch flicks like Bandolero,Fantastic Voyage , Mother ,Jugs & Speed and Myra Breckinridge.Even if you get the DVD-skip the predictable movie and go for the trailer library in the special features.Besides tons of mysoginistic asides-Sinatra lisps @ the homosexual owners of the local Go-Go bar.A relic that needs to be put back in the time capsule. D

---------------------------